In [ ]:
# i think all these will run if you just load the data

treatment_filter = np.zeros_like(df.SubjectTreatment).astype(bool)
for t in ("HVC", "CTRL"):
    treatment_filter |= df.SubjectTreatment == t
    
tsvk_control = get_or_create_Tsvk(df[
    (df.LesionStage == "postlesion")
    & (df.VocalizerSet == "S1")
    & treatment_filter
], k_max=11)
logOR_CTRL = tsvk_control.logOR()
logOR_CTRL

In [ ]:
tsvk_NCM = get_or_create_Tsvk(df[
    (df.LesionStage == "postlesion")
    & (df.VocalizerSet == "S1")
    & (df.SubjectTreatment == "NCM")
], k_max=11)
logOR_NCM = tsvk_NCM.logOR()
logOR_NCM

In [ ]:
def statistic(x, y, axis):

    return np.mean(x, axis=axis) - np.mean(y, axis=axis)

In [ ]:
def d_prime(logOR_CTRL_tsvk, logOR_NCM_tsvk, axis):

    logOR_CTRL_tsvk = logOR_CTRL_tsvk.logOR()
    logOR_NCM_tsvk = logOR_NCM_tsvk.logOR()
    
    diff_logOR = logOR_CTRL_tsvk["logOR"]-logOR_NCM_tsvk["logOR"]
    
    # normalization term
    nNCM = logOR_NCM_tsvk["dof"] + 1
    nCTRL = logOR_CTRL_tsvk["dof"] + 1
    var_NCM = logOR_NCM_tsvk["SE"]**2 * nNCM 
    var_CTRL = logOR_CTRL_tsvk["SE"]**2 * nCTRL
    norm_var = np.sqrt(var_NCM * nNCM + var_CTRL * nCTRL / (nNCM + nCTRL))
    
    # effect size of NCM lesion on memory
    d_prime = (np.sqrt(2) * diff_logOR) / norm_var

    return d_prime

In [ ]:
x = tsvk_NCM
y = tsvk_control

d_prime(x,y,axis = 0)

In [ ]:
x = tsvk_NCM.logOR_by_subjects(0)
y = tsvk_control.logOR_by_subjects(0)

d_prime(x,y,axis = 0)

In [ ]:
# difference of the logORs
diff_logOR = logOR_CTRL["logOR"]-logOR_NCM["logOR"]
    
# normalization term
nNCM = logOR_NCM["dof"] + 1
nCTRL = logOR_CTRL["dof"] + 1
var_NCM = logOR_NCM["SE"]**2 * nNCM 
var_CTRL = logOR_CTRL["SE"]**2 * nCTRL
norm_var = np.sqrt(var_NCM * nNCM + var_CTRL * nCTRL / (nNCM + nCTRL))
    
# effect size of NCM lesion on memory
d_prime = (np.sqrt(2) * diff_logOR) / norm_var

x = tsvk_NCM.logOR_by_subjects(0)['logOR']
y = tsvk_control.logOR_by_subjects(0)['logOR']

stat = (d_prime[0])

res = permutation_test((x, y), stat, vectorized=True,
                       n_resamples=92378, alternative='less')
print(res.stat)
print(res.pvalue)

In [3]:
from scipy.stats import norm
import numpy as np
rng = np.random.default_rng()

x = norm.rvs(size=5, random_state=rng)

y = norm.rvs(size=6, loc = 3, random_state=rng)

In [4]:
x

array([ 0.07984521, -0.46785064, -0.07887365, -0.70315086,  0.34163041])

In [5]:
y

array([3.9677425 , 3.31636434, 3.16185407, 1.6994018 , 3.70496176,
       2.12625357])

In [8]:
x = 11
for n in range(0,x):
  print(n) 

0
1
2
3
4
5
6
7
8
9
10


In [ ]:
def d_prime_by_hand(logOR_by_subj_CTRL, logOR_by_subj_NCM):
    
    """returns a single value for d_prime from two arrays of of logORs, ctrl and NCM
    
        degrees of freedom are HARD CODED and this calculation should not be used elsewhere
        
        logOR_by_subjs input parameters are extracted from tsvk.logOR_by_subjects(k) objects.
        example: tsvkobj = tsvk.logOR_by_subjects(k)    ## must chose a k value
                 logOR_by_subj = tsvkobj['logOR']
    """
    
    mean_NCM, SE_NCM = jackknife(logOR_by_subj_NCM, np.mean)
    mean_ctrl, SE_ctrl = jackknife(logOR_by_subj_CTRL, np.mean)
    
    diff_logOR = mean_ctrl-mean_NCM
    
    # normalization term
    # note that dof is HARD CODED for control versus NCM groups 
    nNCM = 9 + 1    # 9 dof for NCM subjects
    nCTRL = 10 + 1  # 10 dof for ctrl and HVC subjects combined
    var_NCM = SE_NCM**2 * nNCM 
    var_CTRL = SE_ctrl**2 * nCTRL
    norm_var = np.sqrt(var_NCM * nNCM + var_CTRL * nCTRL / (nNCM + nCTRL))
    
    # effect size of NCM lesion on memory
    d_prime = (np.sqrt(2) * diff_logOR) / norm_var

    return d_prime